Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 02: Decision Trees

## Introduction
By now everyone should have found a group. If someone still has none but wants to participate in the course please contact one of the tutors.

This weeks sheet should be solved and handed in before the end of **Sunday, April 24, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups designated tutor or whom ever of us you run into first. Please upload your results to your group's studip folder.

If, during the programming tasks, you run into a `NameError`, make sure that you executed all prior code cells beforehand. Later cells might rely on variables and function from prior cells. To see all currently defined variables you can make use of the `%whos` [magic function](https://github.com/lmmx/devnotes/wiki/IPython-'magic'-function-documentation#whos) anywhere in code cells.

## Assignment 1: Decision Trees [2 Points]
Draw the decision trees for the following boolean functions. Either use pen and paper and bring the results to the feedback session or employ your ASCII artist within below. 

Note: $\oplus := xor$, that means one of the operands has to be true, while the other one has to be false:

$\oplus$ | $B$ | $\neg B$
---------|-----|---------
$A$      |  f  |    t
$\neg A$ |  t  |    f

### a) $\neg A \wedge B$

### b) $A \oplus B$ 

### c) $A \vee (B \wedge C) \vee (\neg C \wedge D)$

### d) $(A \rightarrow (B \wedge \neg C)) \vee (A \wedge B)$

## Assignment 2: Entropy and Information Gain [8 Points]

In many machine learning applications it is crucial to determine which criterions are necessary for a good classification. Decision trees have those criterions close to the root, imposing an order from significant to less significant criterions. One way to select the most important criterion is to compare its information gain or its entropy to others. The following dataset is a hands-on example for this method. 

Consider the following attributes with their possible values:

  * $raining = \{yes, no\}$
  * $tired = \{yes, no\}$
  * $late = \{yes, no\}$
  * $distance = \{short, medium, long\}$

And a training data set consisting of those attributes:

| #  | raining | tired | late | distance | attend_party |
|----|---------|-------|------|----------|--------------|
| 1  | yes     | no    | no   | short    | **yes**      |
| 2  | yes     | no    | yes  | medium   | **no**       |
| 3  | no      | yes   | no   | long     | **no**       |
| 4  | yes     | yes   | yes  | short    | **no**       |
| 5  | yes     | no    | no   | short    | **yes**      |
| 6  | no      | no    | no   | medium   | **yes**      |
| 7  | no      | yes   | no   | long     | **no**       |
| 8  | yes     | no    | yes  | short    | **no**       |
| 9  | yes     | yes   | no   | short    | **yes**      |
| 10 | no      | yes   | no   | medium   | **no**       |
| 11 | no      | yes   | no   | long     | **no**       |
| 12 | no      | yes   | yes  | short    | **no**       |

### a)

Build the root node of a decision tree from the training samples given in the table above by calculating the information gain for all four attributes (raining, tired, late, distance).

$$Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|}Entropy(S_v)$$

$$Entropy(S) = -p_{\oplus} log_{2} p_{\oplus} - p_{\ominus} log_{2} p_{\ominus}$$

$S$ is the set of all data samples. $S_v$ is the subset for which attribute $A$ has value $v$. An example for attribute **tired** with value $yes$ would be:
$$|S_{yes}| = 7, S_{yes}:[1+, 6−]$$

### b)

Perform the same calculation as in **a)** but use the gain ratio instead of the information gain. Does the result for the root node change?

$$GainRatio(S,A) = \frac{Gain(S,A)}{SplitInformation(S,A)}$$

$$SplitInformation(S,A) = - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} log_{2} \frac{S_{v}}{S}$$

## Assignment 3: ID3 algorithm [5 Points]

Implement the following two functions in Python. Take a look at the `assert`s to see how the function should behave. An assert is a condition that your function is required to pass. Most of the conditions here are taken from the lecture slides (ML-03, Slide 12 & 13). Don't worry if you do not get all asserts to pass, just comment the failing ones out.

### a) Entropy

$$Entropy(S) = - \sum_{i=1...c} p_i log_2 p_i$$

In [11]:
from math import log
import numpy as np

def entropy(S):
    """
    Calculate the entropy for a given target value set. 
    S: List of target classes for specific observations.
    """
    prop = np.divide(np.array([S.count(i) for i in np.unique(S)],dtype='float'),len(S))
    return -sum([i*log(i,2) for i in prop])

# See ML-03, Slide 12 & 13
assert entropy([1,1,1,0,0,0]) == 1.0
assert round(entropy([1,1,1,1,0,0,0]), 3) == 0.985
assert round(entropy([1,1,1,1,1,1,0]), 3) == 0.592
assert round(entropy([1,1,1,1,1,1,0,0]), 3) == 0.811
assert round(entropy([2,2,1,1,0,0]), 3) == 1.585
assert round(entropy([2,2,2,1,0]), 3) == 1.371
assert round(entropy([2,2,2,0,0]), 3) == 0.971

### b)  Information Gain

$$Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} Entropy(S_v)$$

In [12]:
from __future__ import division

def gain(S, A):
    """
    Calculates the expected reduction in entropy due to sorting on A.
    S: Target classes for observations in A.
    A: Observations.
    """
    Sum = 0
    for v in np.unique(A):
        Sv   = [x for i,x in enumerate(S) if A[i]==v]
        Sum += len(Sv)/len(S) * entropy(Sv)

    return entropy(S) - Sum




# See ML-03, Slide 12 & 13
assert_S_ = [0,0,1,1,1,0,1,0,1,1,1,1,1,0]
assert round(gain(assert_S_, [1,1,1,1,0,0,0,1,0,0,0,1,0,1]), 3) == .152
assert round(gain(assert_S_, [0,1,0,0,0,1,1,0,0,0,1,1,0,1]), 3) == .048

### c) ID3

In the next two cells we have implemented the ID3 algorithm following the pseudocode from [Wikipedia](https://en.wikipedia.org/wiki/ID3_algorithm#Pseudocode) -- it relies on your two functions from above, `entropy` and `gain`. You do not have to understand the code completly! We recommend that you read through it, making your way from comment to comment, but understanding every line is not necessary for this exercise.

Below the algorithm's cell it is applied to two data sets. Run those and discuss the differences. For which data set is the ID3 algorithm better suited and why? (Enter your answer in the cell below the code.)

In [13]:
from collections import namedtuple

class Node(namedtuple('Node', 'label children')):
    """
    A small node representation with a pretty string representation.
    """
    def __str__(self, level=0):
        return_str ='{}{!s}\n'.format(' ' * level * 4, self.label)
        for child in self.children:
            return_str += child.__str__(level + 1)
        return return_str

In [14]:
from collections import Counter

def id3(examples, attributes, target_attribute = None): 
    """
    Calculate a tree of Nodes (fields: label [string], children [list]) 
    using the ID3 algorithm found as pseudocode on Wikipedia. Including comments (currently).
    """
    # Create a root node for the tree
    # If all examples are positive, Return the single-node tree Root, with label = 
    # If all examples are negative, Return the single-node tree Root, with label = -.
    if all(target == examples['targets'][0] for target in examples['targets']):
        return Node('Result: {!s}'.format(examples['target_names'][examples['targets'][0]]), [])
    
    # If number of predicting attributes is empty, then Return the single node tree Root,
    # with label = most common value of the target attribute in the examples.
    if len(attributes) == 0:
        attr = Counter(data_sample[target_attribute] for data_sample in examples['data']).most_common(1)
        return Node('Attribute: {!s}, {!s} occurences'.format(examples['attributes'][target_attribute], attr), [])
    
    # A <-- The Attribute that best classifies examples.
    gains = [gain(examples['targets'], [r[attribute] for r in examples['data']]) for attribute in attributes]
    attribute = attributes[gains.index(max(gains))]

    # Create a root node for the tree (2)
    # Decision Tree attribute for Root = A.
    root = Node('Attribute: {!s} (gain {!s})'.format(examples['attributes'][attribute], round(min(gains), 4)), [])
    
    # Otherwise Begin
    # For each possible value, vi, of A,
    for vi in set(data_sample[attribute] for data_sample in examples['data']):
        # Add a new tree branch below Root, corresponding to the test A = vi.
        child = Node('Value: {!s}'.format(vi), [])
        root.children.append(child)
        
        # Let Examples(vi) be the subset of examples that have the value vi for A
        vi_indices = [idx for idx, data_sample in enumerate(examples['data']) if data_sample[attribute] == vi]
        examples_vi = dict(examples)
        examples_vi['data'] = [examples['data'][i] for i in vi_indices]
        examples_vi['targets'] = [examples['targets'][i] for i in vi_indices]
        
        # If Examples(vi) is empty
        if len(examples_vi['data']) == 0:
            # Then below this new branch add a leaf node with label = most common target value in the examples
            attr = Counter(examples_vi['targets']).most_common(1)
            label = 'Attribute: {!s}, {!s} occurences'.format(examples['attributes'][target_attribute], attr)
            child.children.append(Node(label, []))
        else:
            # Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {A})
            child.children.append(
                id3(examples_vi, \
                    [attribute_ for attribute_ in attributes if not attribute_ == attribute], \
                    attribute)
            )
    # End

    # Return root
    return root

This code runs the ID3 algorithm on the party data set which you already know from assignment 2.

In [15]:
import json

with open('party.json', 'r') as party_file:
    party = json.load(party_file)

# Make sure our gain function handles the data set as expected.
assert round(gain(party['targets'], [r[2] for r in party['data']]), 3) == 0.252

# Apply ID3 algorithm
tree_party = id3(party, list(range(len(party['attributes']))))

print(tree_party)

Attribute: late (gain 0.0933)
    Value: yes
        Result: no
    Value: no
        Attribute: distance (gain 0.5488)
            Value: medium
                Attribute: tired (gain 0.0)
                    Value: yes
                        Result: no
                    Value: no
                        Result: yes
            Value: short
                Result: yes
            Value: long
                Result: no



This code runs the ID3 algorithm on the famous iris flowser data set, which you will hear more about in assignment 4 below.

In [16]:
import json

with open('iris.json', 'r') as iris_file:
    iris = json.load(iris_file)

# Make sure our gain function handles the data set as expected.
assert round(gain(iris['targets'], [r[2] for r in iris['data']]), 3) == 1.446

# Apply ID3 algorithm
tree_iris = id3(iris, list(range(len(iris['attributes']))))

print(tree_iris)

Attribute: petal length (gain 0.5166)
    Value: 1.4
        Result: Iris-setosa
    Value: 1.5
        Result: Iris-setosa
    Value: 1.6
        Result: Iris-setosa
    Value: 1.7
        Result: Iris-setosa
    Value: 1.0
        Result: Iris-setosa
    Value: 1.1
        Result: Iris-setosa
    Value: 1.2
        Result: Iris-setosa
    Value: 1.3
        Result: Iris-setosa
    Value: 1.9
        Result: Iris-setosa
    Value: 3.6
        Result: Iris-versicolor
    Value: 3.7
        Result: Iris-versicolor
    Value: 3.5
        Result: Iris-versicolor
    Value: 6.3
        Result: Iris-virginica
    Value: 3.3
        Result: Iris-versicolor
    Value: 3.0
        Result: Iris-versicolor
    Value: 6.0
        Result: Iris-virginica
    Value: 3.8
        Result: Iris-versicolor
    Value: 3.9
        Result: Iris-versicolor
    Value: 4.5
        Attribute: sepal length (gain 0.5436)
            Value: 5.4
                Result: Iris-versicolor
            Value: 5.6
       

## Assignment 4: Decision Trees on Iris Flowers [5 Points]

In this exercise we are going to examine and compare two decision trees that were generated from the iris flower data set ([Wikipedia](https://en.wikipedia.org/wiki/Iris_flower_data_set)) where three variations of the iris flower are quantified. The Iris data set is a classical example of a labeled dataset, i.e. every sample consists of two parts: features and labels. There are four features per sample in this data set (sepal length ($x_1$), sepal width ($x_2$), petal length ($x_2$) and petal width ($x_4$) in cm) and a corresponding label (Iris Setosa, Iris Versicolour, Iris Virginica). These samples are by nature **noisy**, no matter how carefully the measurement was taken - slight deviation from the actual length **cannot be avoided**. We want to learn how the features are related to the label so that we could (in the future) predict the label of a new sample automatically. One way to obtain such a `classifier` is to train a decision tree on the data.

Here are two decisions tree generated by the data set. We will now take a closer look.

### Tree 1:

### Tree 2:

### a)

What does it mean that the features $x1$ and $x2$ do not appear in the decision trees?

### b)
With which method from the lecture might the second tree have been generated from the first one? Explain the procedure.

### c)
After training the tree we can calculate the accuracy, i.e. the percentage of the training set that is classified correctly. Although the first tree was trained on the data set until no improvement of the accuracy was possible, its accuracy is *only* 98%. Explain why it is not 100 %

### d)

Tree 2 only has a 96% accuracy on the training set. Why might this tree still be preferable over tree 1?